In [2]:
import psycopg2
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# Constants
WINDOW_LENGTH = 24  # Adjust as needed
MIN_LOS_ICU = 48    # Adjust as needed
MAX_TIMESTEPS = 10  # Set an upper limit for the number of timesteps
CLIENT_COUNT = 4    # Number of clients

# Connect to db
conn = psycopg2.connect(host='localhost', port=5432, dbname='mimic', user='zainab', password='password')
cur = conn.cursor()

# Read vital signs
vitals_query = f'SELECT * FROM mimiciii.vitals_windowed_{WINDOW_LENGTH}h;'
vitals = pd.read_sql_query(vitals_query, conn)

# Read in labs values
labs_query = f'SELECT * FROM mimiciii.labs_windowed_{WINDOW_LENGTH}h;'
labs = pd.read_sql_query(labs_query, conn)

# Close the cursor and connection
cur.close()
conn.close()

# Convert datetime columns to seconds
for col in vitals.select_dtypes(include=['datetime64', 'timedelta64']).columns:
    vitals[col] = vitals[col].astype(int) / 10**9  # Convert to seconds

for col in labs.select_dtypes(include=['datetime64', 'timedelta64']).columns:
    labs[col] = labs[col].astype(int) / 10**9  # Convert to seconds

# Convert categorical columns to numeric using one-hot encoding
vitals = pd.get_dummies(vitals, drop_first=True)
labs = pd.get_dummies(labs, drop_first=True)

# Merge the vitals and labs data on the common key (icustay_id)
merged_data = pd.merge(vitals, labs, on='icustay_id', suffixes=('_vitals', '_labs'), how='inner')

# Ensure the labels are the same in both datasets
assert all(merged_data['label_death_icu_vitals'] == merged_data['label_death_icu_labs']), "Mismatch in labels between vitals and labs"

# Drop one of the label columns
merged_data = merged_data.drop(columns=['label_death_icu_labs'])

# Rename the remaining label column for consistency
merged_data = merged_data.rename(columns={'label_death_icu_vitals': 'label_death_icu'})

vitals_features = merged_data.filter(like='_vitals')
labs_features = merged_data.filter(like='_labs')
labels = merged_data['label_death_icu']

# Convert to NumPy arrays
X_vitals = vitals_features.to_numpy()
X_labs = labs_features.to_numpy()
y = labels.to_numpy()

# Calculate the number of timesteps that divides the total features without a remainder
def find_timesteps(total_features, max_timesteps):
    for timesteps in range(max_timesteps, 0, -1):
        if total_features % timesteps == 0:
            return timesteps
    return 1  # Fallback to 1 if no valid timesteps found

# Find suitable timesteps for vitals and labs
timesteps_vitals = find_timesteps(X_vitals.shape[1], MAX_TIMESTEPS)
timesteps_labs = find_timesteps(X_labs.shape[1], MAX_TIMESTEPS)

# Calculate number of features per timestep
num_features_vitals = X_vitals.shape[1] // timesteps_vitals
num_features_labs = X_labs.shape[1] // timesteps_labs

# Reshape data into 3D arrays (samples, timesteps, features)
X_vitals = X_vitals.reshape((-1, timesteps_vitals, num_features_vitals))
X_labs = X_labs.reshape((-1, timesteps_labs, num_features_labs))

# Split data among clients
def split_data(X_vitals, X_labs, y, num_clients):
    vitals_splits = np.array_split(X_vitals, num_clients)
    labs_splits = np.array_split(X_labs, num_clients)
    labels_splits = np.array_split(y, num_clients)
    return vitals_splits, labs_splits, labels_splits

X_vitals_splits, X_labs_splits, y_splits = split_data(X_vitals, X_labs, y, CLIENT_COUNT)

# Define input shapes
vitals_spec = tf.TensorSpec(
    shape=(timesteps_vitals, num_features_vitals),
    dtype=tf.dtypes.float64,
    name='vitals'
)
labs_spec = tf.TensorSpec(
    shape=(timesteps_labs, num_features_labs),
    dtype=tf.dtypes.float64,
    name='labs'
)

# Define the custom F1-score metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Define the model architecture
def create_model():
    # Vital channel
    inputs_vitals = tf.keras.Input(shape=vitals_spec.shape, name='Input_vitals')
    mask_vitals = tf.keras.layers.Masking(mask_value=-2., name='mask_vitals')(inputs_vitals)
    GRU_layer1_vitals = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer1_vitals')(mask_vitals)
    GRU_layer2_vitals = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer2_vitals')(GRU_layer1_vitals)
    GRU_layer3_vitals = tf.keras.layers.GRU(16, return_sequences=False, name='GRU_layer3_vitals')(GRU_layer2_vitals)
    normalized_vitals = tf.keras.layers.BatchNormalization(name='BatchNorm_vitals')(GRU_layer3_vitals)

    # Labs channel
    inputs_labs = tf.keras.Input(shape=labs_spec.shape, name='Input_labs')
    mask_labs = tf.keras.layers.Masking(mask_value=-2., name='mask_labs')(inputs_labs)
    GRU_layer1_labs = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer1_labs')(mask_labs)
    GRU_layer2_labs = tf.keras.layers.GRU(16, return_sequences=True, name    ='GRU_layer2_labs')(GRU_layer1_labs)
    GRU_layer3_labs = tf.keras.layers.GRU(16, return_sequences=False, name='GRU_layer3_labs')(GRU_layer2_labs)
    normalized_labs = tf.keras.layers.BatchNormalization(name='BatchNorm_labs')(GRU_layer3_labs)

    # Concatenation of both branches
    merge = tf.keras.layers.Concatenate()([normalized_vitals, normalized_labs])

    FCL1 = tf.keras.layers.Dense(16, name='FCL1')(merge)
    FCL2 = tf.keras.layers.Dense(16, name='FCL2')(FCL1)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(FCL2)

    model = tf.keras.Model(inputs=[inputs_vitals, inputs_labs], outputs=outputs, name='RNN_model')
    return model

# Train model on each client's data and aggregate the weights
def federated_training(X_vitals_splits, X_labs_splits, y_splits, num_clients, global_epochs, local_epochs, batch_size):
    # Initialize the global model
    global_model = create_model()
    global_model.compile(optimizer='adam',
                         loss='binary_crossentropy',
                         metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])

    for global_epoch in range(global_epochs):
        print(f"Global Epoch {global_epoch+1}/{global_epochs}")

        # Initialize list to store client models
        client_models = []

        for client in range(num_clients):
            print(f" Training on client {client+1}/{num_clients}")
            client_model = create_model()
            client_model.compile(optimizer='adam',
                                 loss='binary_crossentropy',
                                 metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])
            
            # Set the client model weights to the global model weights
            client_model.set_weights(global_model.get_weights())

            # Train the client model
            client_model.fit(x=[X_vitals_splits[client], X_labs_splits[client]],
                             y=y_splits[client],
                             epochs=local_epochs,
                             batch_size=batch_size,
                             verbose=0)
            
            # Append trained client model to the list
            client_models.append(client_model)

        # Aggregate the client model weights
        average_weights = []
        for weights_list in zip(*[client.get_weights() for client in client_models]):
            average_weights.append(np.mean(weights_list, axis=0))

        # Update global model with the aggregated weights
        global_model.set_weights(average_weights)

    return global_model

# Perform federated training
global_model = federated_training(X_vitals_splits, X_labs_splits, y_splits, CLIENT_COUNT, global_epochs=3, local_epochs=5, batch_size=32)

# Evaluate the global model
evaluation = global_model.evaluate(x=[X_vitals, X_labs], y=y, verbose=0)

# Print out the evaluation metrics
print("Evaluation Metrics:")
print(f"Accuracy: {evaluation[1]}")
print(f"Precision: {evaluation[2]}")
print(f"Recall: {evaluation[3]}")
print(f"F1 Score: {evaluation[4]}")



2024-05-21 11:04:57.619088: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 11:04:57.622191: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 11:04:57.667251: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 11:04:58.369236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_47409/859392504.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vitals = pd.read_sql_query(vitals_q